# Gradient Descent

Optimizing functions algorithmically

In [1]:
from linalg import Vector, dot, distance, add, scalar_multiply, add, vector_mean
from typing import Callable
import random
import numpy as np

In [2]:
def sum_of_squares(xs: Vector) -> float:
    """
    Return the sum of the square of each element in xs
    """
    # this is equivalent to x dot x
    return dot(x, x)

Consider a function (e.g. a loss function) which reduces a vector to a meaningful float. The main idea of gradient descent is to algorithmically find the inputs that minimize this reducing function

### Terms:
- **Gradient**: The vector of partial deriviates for a vector relative to a function. E.g. if `y = sum_of_squares(xs)`, the gradirent is `dy/dxs` or `[dy/dx_0, dy/dx_1, ... dy/dx_n]`

#### Estimating the Gradient

In [3]:
def difference_quotient(f: Callable[[float], float], x: float, h: float) -> float:
    return (f(x + h) - f(x)) / h


This is the definition of a gradient for a single variable x and function f(x). We can estimate the gradient by just choosing a very small h (e.g. 10**-6). We can also do this for partial-derivatives in a vector calculus setting for f(xs):

In [4]:
def partial_diff_quotient(f: Callable[[Vector], float], xs: Vector, i: int, h: float) -> float:
    w = [x_j + (h if i == j else 0) for j, x_j in enumerate(xs)]  # single out and add h to just the ith element of xs
    return (f(w) - f(xs)) / h  # reflects only the change we made to the ith variable

In [5]:
def estimate_gradient(f: Callable[[Vector], float], xs: Vector, h: float = 10**-4) -> Vector:
    """
    Estimate the gradient of f with respect to xs by computing partial diff quotients element-wise
    """
    # note this is expensive and why auto-grad libraries mathematically compute most derivatives
    return [partial_diff_quotient(f, xs, i, h) for i in range(len(xs))]

### Using the Gradient

For sum of squares, it is obvious the minimum overall is achieved by a vector of zeros. In some cases though, the minimum(s) may not be obvious, so we'll use it as an example to evaluatate our algorithm

In [6]:
def gradient_step(xs: Vector, gradient: Vector, step_size: float) -> Vector:
    """
    Moves `step_size` along the gradient of f w.r.t. xs, returning a input
    """
    assert len(xs) == len(gradient)
    update = scalar_multiply(step_size, gradient)
    return add(xs, update)

def sum_of_squares_gradient(xs: Vector) -> Vector:
    """
    We know the partial-derivative for a sum of squares is just 2*`the_term`
    """
    return [2*x for x in xs]    

In [7]:
# Now lets verify with an experiment
xs = [random.uniform(-10, 10) for i in range(10)]

for i in range(1, 100000):
    grad = sum_of_squares_gradient(xs)
    xs = gradient_step(xs, grad, -1 * (10**-4))

assert(distance(xs, np.zeros(10)) < 10**-6) # we should have gotten very close to zero
print(xs)

[-1.1522938445231776e-08, 1.5514131039526043e-09, 9.878463039780581e-09, 1.941546825631437e-08, -1.5841937294312695e-08, 1.110593240200334e-09, 1.524930648300702e-08, 9.542884342392084e-09, -1.373269285380657e-08, -1.2214808198697881e-08]


In [27]:
# x ranges from (-50, 49), y = 20 * x + 5: we'll use gradient descent to fit parameters to this, 
# as if we had no idea y = 20*x + 5
data = [(x, 0*(x**2) + 20*x + 5) for x in range(-10, 10)]

In [28]:
# we think f(x) is a polynomial, and want to compute a gradient using coefficients ws
def linear_gradient_mse(x: float, y: float, ws: Vector) ->  float:
    predicted = sum([w * (x**i) for i, w in enumerate(ws)]) # our weights are coefficients to the polynomial
    target = y 
    error = predicted - target  
    grad = [2 * error * (x**i) for i in range(len(ws))]
    return grad    

In [29]:
weights_linear = [random.uniform(-1, 1) for i in range(2)]
lr = 10**-3
for epoch in range(5000):
    # compute mean of gradients
    mean_grad = vector_mean([linear_gradient_mse(x, y, weights_linear) for x, y in data])
    weights_linear = gradient_step(weights_linear, mean_grad, -1 * lr)
    if epoch % 500 == 0 or epoch == 4999:
        print(epoch, weights_linear)  # second near 20, first near 5

0 [0.13139582251931237, 0.0022031918474523238]
500 [86.23102134374972, 15.219123569003385]
1000 [118.19749667312084, 15.710799151759428]
1500 [130.0364143476073, 15.892893257751007]
2000 [134.42100696406288, 15.960332570582063]
2500 [136.04485915569532, 15.985309001808963]
3000 [136.6462595983212, 15.99455912745]
3500 [136.86899076452863, 15.997984950122493]
4000 [136.95148018236725, 15.999253717860224]
4500 [136.9820304828539, 15.999723611292032]
4999 [136.99333168018492, 15.999897434734441]


In [38]:
data = [(x, -4*(x**2) + 20*x + 5) for x in range(-10, 10)]
# now lets attempt to fit a cubic to a linear: the 2nd and 3rd degree terms (last 2) should near zero over time
weights_cubic = [random.uniform(-1, 1) for i in range(3)]
print(weights_cubic)
lr = 2 * 10**-4 # slow down for visibility
for epoch in range(25000):
    # compute mean of gradients
    mean_grad = vector_mean([linear_gradient_mse(x, y, weights_cubic) for x, y in data])
    # print('mean_grad', mean_grad)
    weights_cubic = gradient_step(weights_cubic, mean_grad, -1 * lr)
    if epoch % 1000 == 0 or epoch == 9999:
        print(epoch, weights_cubic)  # second near 20, first near 5

[-0.3491176762602788, 0.777818273803206, 0.30822641485799473]
0 [-0.40855269949411116, 1.120490213696151, -3.508505750536359]
1000 [0.6284423399413209, 20.044456344519602, -3.926866432310565]
2000 [1.3407984513870967, 20.037248374177395, -3.9387828207464475]
3000 [1.9370740542144136, 20.0311786630212, -3.948758305827073]
4000 [2.4361851281666964, 20.02609802566323, -3.9571082618699127]
5000 [2.8539648645185443, 20.021845290256813, -3.964097572699071]
6000 [3.203666398335672, 20.01828554820825, -3.9699479586910393]
7000 [3.4963832347766157, 20.015305874600223, -3.974845010359171]
8000 [3.741401165927028, 20.01281174589952, -3.978944075800883]
9000 [3.9464928419479857, 20.010724041407702, -3.982375188771322]
9999 [4.1180074712879415, 20.00897813016922, -3.9852445693369587]
10000 [4.118164341154576, 20.00897653333247, -3.9852471937156975]
11000 [4.261861560913138, 20.00751378585791, -3.9876511986177747]
12000 [4.382142976650603, 20.00628939656632, -3.989663465198489]
13000 [4.482824249371

In [36]:
data = [(x, -4*(x**2) + 20*x + 5) for x in range(-3, 3)]
# now lets attempt to fit a cubic to a linear: the 2nd and 3rd degree terms (last 2) should near zero over time
weights_cubic = [random.uniform(-1, 1) for i in range(4)]
print(weights_cubic)
lr = 2 * 10**-4 # slow down for visibility
for epoch in range(35000):
    # compute mean of gradients
    mean_grad = vector_mean([linear_gradient_mse(x, y, weights_cubic) for x, y in data])
    # print('mean_grad', mean_grad)
    weights_cubic = gradient_step(weights_cubic, mean_grad, -1 * lr)
    if epoch % 1000 == 0 or epoch == 9999:
        print(epoch, weights_cubic)  # second near 20, first near 5

[-0.9445286908931119, 0.20193238324695773, -0.05159987214657358, 0.25477570730816756]
0 [-0.9506532034955649, 0.23097486963077318, -0.10585035870341508, 0.44523466729237776]
1000 [-0.8955570292683016, 3.7929874249319586, -0.4977688367670486, 2.9828015403551373]
2000 [-0.8032004393067043, 6.319507430906873, -0.7992220599917735, 2.5614234055428984]
3000 [-0.6181605240651165, 8.410723638222807, -1.1075557655359858, 2.1989862987607056]
4000 [-0.3649318214075337, 10.150708102621218, -1.3908745936785503, 1.8929546305811762]
5000 [-0.06836195098100008, 11.603076834135482, -1.6492020082913355, 1.6340122989864045]
6000 [0.25265234572006884, 12.819186851395765, -1.8840895351402445, 1.414224016339089]
7000 [0.5841145165478869, 13.84071832840359, -2.0971943735696628, 1.2270784791195215]
8000 [0.915831749119824, 14.701565335565274, -2.2901705880970322, 1.067231446233203]
9000 [1.240540997891608, 15.429343791065415, -2.4646322610081817, 0.9302860180073766]
9999 [1.552917261263811, 16.046032417903767

## fin
I spent way too much time just getting this to work and while the it can technically fit things correctly for arbitrary polynomials, the settings are extremely sensitive to both learning rate and batch size, causing major issues with overflow. Not worth digging too far into that, but I made changes to `vector_mean` that might help. Skipping **SGD** and **Batch-SGD** because they follow from these.